In [3]:
#Import requirements:
import cv2 as cv


# The Pong Project
Christian Kuate, Aissata Sylla, Raphaela NKOUDOU, Paul Béglin

### Présentation synthétique du projet


### Répartition des tâches
Nous avons décidé de séparer le projet en 2 parties: 
1. le jeu (avec comme controles les "UP ARROW" "DOWN ARROW" normaux, (que Christian et Aissata auraient du faire)
2. les contrôles fait par une main, grâce à OpenCV : fait par Paul 
3. le Menu fait par Raphaela

Puis nous allons intégrer ces 3 parties ensemble

### Initialisation
#### Git & GitHub
Afin d'apprendre à utiliser git (un des outils les plus importants dans notre carrière de développeurs) nous allons tout faire sur GitHub en essayant d'adopter les bonnes pratiques, comme créer une branche pour chaque feature avant de merge avec main.

#### Environnement Virtuel Python
De plus, à cause de problèmes d'environement qui empêchait de faire fonctionner OpenCV, nous avons créé des environnements virtuels afin de séparer la version Python qu'utilise le système et la version dont nous avons besoin avec toutes nos dépendences. On a notamment utilisé Conda.

...



In [ ]:
#Premier tests caméra avec opencv et cvzone Hand Tracking
#Attention, dans Jupyternotebooks on ne peut pas sortir de GUI, donc afin d'avoir quelque chose qui s'affiche inline
#j'ai changé le code pour utiliser matplotlibe et avoir chaque image qui s'affiche inline... 
#MAIS il faut arrêter le processus rapidement, sinon vous allez avoir pleins d'images 

import cv2 as cv
from cvzone.HandTrackingModule import HandDetector
from matplotlib import pyplot as plt

liveVideo = cv.VideoCapture(0)
handDetector = HandDetector(detectionCon = 0.8)

while True:
    ret, frame = liveVideo.read()

    #Show the image with matplotlib
    plt.imshow(frame)
    plt.show()
    #cv.imshow('frame', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):    #waitKey(1) & 0xFF is a bitwise operation to only keep the last 8 bits and compare it to ord('q')
        break

# After the loop release the cap object
liveVideo.release()
# Destroy all the windows
cv.destroyAllWindows()

On utilise la point de l'index comme réference pour le mouvement (le point numéro 8 de la main sur OpenCV)
c'est pourquoi on appelle: 

```yCoordinate = lmList1[8][1:2]```

Pour avoir ses coordonnées

In [ ]:
while True:
    ret, frame = liveVideo.read()
    frame = cv.flip(frame, 1)
    hands, frame = handDetector.findHands(frame, flipType=False)
#for each hand we'Ll have info Like Hand--›dict{lmList,boundinqboundary, img)
    if hands:
        hand1=hands[0]                          #gives us first hand
        lmList1=hand1["lmList"]                 #List of 21 Landmarks
        bbox1=hand1["bbox"]                     #x,y,w,h of bounding box
        centerPoint1=hand1[ "center" ]          #center of the hand cx, cy
        handType1=hand1["type"]                 #Left or right
        finger1=handDetector.fingersUp(hand1)
        
        #needs something to find the distance / x,y positions 
        #length, info, frame=handDetector.findDistance(lmList1[8], lmList1[12])
        yCoordinate = lmList1[8][1:2]       
        #yCoordinate = tuple(yCoordinate) #careful change data type to tuple with tuple() if doing a circle bounding box
        #cv2.circle() ...
        print(yCoordinate)

LA CRÉATION DE LA PAGE D'ACCEUIL
Pour réaliser la page d'accueil nous avons utilisé la bibliothéque pygame en utilisant le concept de multi-fenetres. A cet effet nous avons donc créer cinq fonctions principales qui constituent les quatres boutons de l'interface et la fonction main qui va se charger d'afficher ces boutons à l'écran.
le principe est simple, le clic d'un bouton correspond à l'appel de la fonction
Egalement, pour generer ces boutons à l'ecran, nous avons opté pour la création d'une classe BUTTON. Ce qui rend la géneration des boutons facile et non redondante.

Importation de pygame et Configuration de la fenetre du menu d'accueil


In [ ]:
import pygame,sys
LARGEUR = 1280
HAUTEUR = 900

pygame.init()

fenetre = pygame.display.set_mode((LARGEUR, HAUTEUR))
pygame.display.set_caption("Menu")
icone = pygame.image.load('image jeu/ICONE.png')
pygame.display.set_icon(icone)

background = pygame.image.load('image jeu/BACK2.jpg')

Definition de la police du menu
cette fonction va retourner la police principale du menu qui sera utilisé au niveau des boutons

In [ ]:
def get_font(size): 
    return pygame.font.Font('image jeu/font.ttf', size)

La classe BUTTON:
    le def _init_
    Elle posséde un constructeur qui va initialiser les différents attributs des boutons(image, position du bouton, text, la police, la couleur, et la couleur secondaire) et trois méthodes

In [ ]:
class Button():
	def __init__(self, image, pos, text_input, font, base_color, hovering_color):
		self.image = image
		self.x_pos = pos[0]
		self.y_pos = pos[1]
		self.font = font
		self.base_color, self.hovering_color = base_color, hovering_color
		self.text_input = text_input
		self.text = self.font.render(self.text_input, True, self.base_color)
		if self.image is None:
			self.image = self.text
		self.rect = self.image.get_rect(center=(self.x_pos, self.y_pos))
		self.text_rect = self.text.get_rect(center=(self.x_pos, self.y_pos))

    la methode update
    elle permet d'afficher le bouton à l'ecran d'accueil une fois qu'elle est appelée

In [ ]:
def update(self, screen):
		if self.image is not None:
			screen.blit(self.image, self.rect)
		screen.blit(self.text, self.text_rect)

la methode checkForInput
elle permet de vérifier si u utilisateur a cliqué sur un bouton

In [ ]:
def checkForInput(self, position):
		if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
			return True
		return False

la methode changeColor
Elle permet de changer de couleur en réaction au clic de la souris

In [ ]:
def changeColor(self, position):
		if position[0] in range(self.rect.left, self.rect.right) and position[1] in range(self.rect.top, self.rect.bottom):
			self.text = self.font.render(self.text_input, True, self.hovering_color)
		else:
			self.text = self.font.render(self.text_input, True, self.base_color)

La fonction Play
Il s'agit du main.py
C'est grace à elle que le jeu se lance. Nous reviendrons dessus un peu plus bas

La fonction aide
C'est dans cette fonction que s'effectue l'affichage des régles du jeu.
Une fois que t'utilisateur clique sur le bouton help, une fenetre différente du menu d'accueil s'ouvre et présente les régles pour etre mesure de jouer

In [ ]:
def aide():
    while True:
        HELP_MOUSE_POS = pygame.mouse.get_pos()

        fenetre.fill("white")
        titleFont  =  pygame.font.Font( 'image jeu/font.ttf' , 100 )
       
        texteRegles  =  pygame.font.Font( 'image jeu/font.ttf'  , 25 )

        # zones cliquables
        

        #affichage à l'écran
        fenetre.blit ( titleFont . render ( "Règles" , True , ( 0 , 0 , 0 )), ( 370, 30 ))
        bouton_back = Button(image=None, pos=(640, 540), 
                            text_input="BACK", font=get_font(75), base_color="Black", hovering_color="Green")
        fenetre.blit ( texteRegles . render ( 'Le ping pong est une sorte de tennis de table ' , True , ( 0 , 0 , 0 )), ( 20 , 150  ))
        fenetre.blit ( texteRegles . render ( "en mouvement constant. L'un gagne si l 'autre ne  ", True , ( 0 , 0 , 0 )), ( 20, 200 ))
        fenetre.blit ( texteRegles . render ( "parvient pas à intercepter la balle de l'autre" , True , ( 0 , 0 , 0 )), ( 20 , 250 ))
        fenetre.blit ( texteRegles . render ( 'ici les déplacements sont controlés par les mou' , True , ( 0 , 0 , 0 )), ( 20 , 300 ))
        fenetre.blit ( texteRegles . render ( 'vements du bras. si le joueur souhaite se déplacer' , True , ( 0 , 0 , 0 )), ( 20 , 350 ))
        fenetre.blit ( texteRegles . render ( "vers le haut, il souleve la main et s'il souhaite se"  , True , ( 0 , 0 , 0 )), ( 20 , 400 ))
        fenetre.blit ( texteRegles . render ( 'deplacer vers le bas, il baisse sa main' , True , ( 0 , 0 , 0 )), ( 20 , 450 ))
    

        

        bouton_back.changeColor(HELP_MOUSE_POS)
        bouton_back.update(fenetre)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if bouton_back.checkForInput(HELP_MOUSE_POS):
                  main_menu()

        pygame.display.update()    


la fonction main menu()
C'est dans cette fonction qu'on crée nos objets boutons 

In [ ]:
def main_menu():
    while True:
        fenetre.blit(background, (0, 0))

        MENU_MOUSE_POS = pygame.mouse.get_pos()

        MENU_TEXT = get_font(100).render("MAIN MENU", True, (182, 143, 64))
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))

        Bouton_play = Button(image=pygame.image.load("image jeu/Play Rect.png"), pos=(640, 250), 
                            text_input="PLAY", font=get_font(75), base_color=(215,252,212), hovering_color="White")
        Bouton_help = Button(image=pygame.image.load("image jeu/aide Rect.png"), pos=(640, 400), 
                            text_input="HELP", font=get_font(75), base_color=(215,252,212), hovering_color="White")
        Bouton_score = Button(image=pygame.image.load("image jeu/aide Rect.png"), pos=(640, 550), 
                            text_input="SCORE", font=get_font(75), base_color=(215,252,212), hovering_color="White")                   
        Bouton_quitter = Button(image=pygame.image.load("image jeu/Quit Rect.png"), pos=(640, 700), 
                            text_input="QUIT", font=get_font(75), base_color=(215,252,212), hovering_color="White")

        fenetre.blit(MENU_TEXT, MENU_RECT)

        for button in [Bouton_play,Bouton_help ,Bouton_score,Bouton_quitter ]:
            button.changeColor(MENU_MOUSE_POS)
            button.update(fenetre)

Dans la suite, on aura besoin de connaire la position du curseur de la souris pour detrrminer l'action qu'il faut effectuer
Nous allons donc utiliser la fonction get_pos() pour cela

In [ ]:
MENU_MOUSE_POS = pygame.mouse.get_pos()

Pour le moment, nos différents élements son chargés mais n'apparaissent pas sur l'écran et ce pour plusieurs raisons. Premiérement il faut definir les différents evenements qui vont s'effectuer sur la fenetre notamment le clic de la souris sur un bouton ou encore la fermeture de l'écran.
Acet effet nous alllons récupérer les différents événements de la fenetre grâce à la fonction pygame.event.get() et determiner ce qu'il faut faire en fonction du type d'action

In [ ]:
for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if Bouton_play.checkForInput(MENU_MOUSE_POS):
                    play()
                if Bouton_help.checkForInput(MENU_MOUSE_POS):
                    aide()
                if Bouton_score.checkForInput(MENU_MOUSE_POS):
                    score()
                if Bouton_quitter.checkForInput(MENU_MOUSE_POS):
                    pygame.quit()
                    sys.exit()

Aprés l'execution du code, on constate que les différents élements crées ne s'affichent toujours pas. Il faut donc effectuer une mise à jour afin de rafraichir ces elements au niveau de l'ecran.

In [ ]:
pygame.display.update()

Le fichier main
Il contient le jeu proprement dit. De l'affichage des différents joueurs à l'execution du jeu en lui-même. 
Il contient les différentes classes qui constituent les éléments graphiques et les fonctionalités propores au jeu, les événements et les conditions d'executions du jeu. 


La classe Paddle
Elle représente la classe à partir de laquelle on va instancier notre objet joueur(le joueur utilisateur) 
Elle  possede trois attributs (la hauteur, la largeur et la position en x sur la fenetre)
Rect permet de geberer une surface rectangulaire qui va me permettre de déplacer mon joeur

In [ ]:
class Paddle:
    def __init__(self, x, w, h):
        self.w = w
        self.h = h
        self.x = x
        self.y = window_height / 2
        self.rect = pygame.Rect(self.x, self.y, self.w, self.h)

Sa methode principale est la methode move lui permettant d'effectuer les mouvements verticaux via la camera

In [ ]:
def detectMove(self, cy):
        self.rect.y = int(cy)
        self.draw()

le joueur a bien besoin d'un adversaire!!
De ce fait nous avons crée une seconde classe AutoPaddle et donc notre joueur ordinateur sera une instance de cette classe
En plus des attributs d'un joueur normal, elle va prendre les attributs ball et vitesse
pourquoi direz-vous
je vous l'explique plus bas

In [ ]:
class AutoPaddle(Paddle):
    def __init__(self, x, w, h, speed, ball):
        super().__init__(x, w, h)
        self.speed = speed
        self.ball = ball

Comme méthode, notre joueur ordi doit etre capable de se déplacer.
Il se pose donc le probleme du comment déplacer ce joueur ordinateur. Dans notre cas, il va se copier sur les mouvements de la balle
C'est pour cette raison qu'elle prend en argument la vitesse et les mouvements de la table

In [ ]:
def move(self):
        if self.ball.dir_x == 1:
            if self.rect.y + self.rect.h/2 < self.ball.rect.bottom:
                self.rect.y += self.speed
            if self.rect.y + self.rect.h/2 > self.ball.rect.bottom:
                self.rect.y -= self.speed

La classe game
C'est dans cette classe qu'on va charger notre jeu
Premiérement on va créer les différents objets de notre jeu dans le def _init_

In [ ]:
class Game:
    def __init__(self, line_thickness=10, speed=5):
        self.line_thickness = line_thickness
        self.speed = speed
        ball_x = window_width / 2
        ball_y = window_height / 2
        ball_w = self.line_thickness
        ball_h = self.line_thickness
        self.ball = Ball(ball_x, ball_y, ball_w, ball_h, self.speed)
        self.paddles = {}
        paddle_x = 20
        paddle_w = self.line_thickness
        paddle_h = 50
        self.paddles['user'] = Paddle(paddle_x, paddle_w, paddle_h)
        self.paddles['computer'] = AutoPaddle(window_width - paddle_x - 10, paddle_w, paddle_h, self.speed, self.ball)
        self.score = ScoreBoard()

Ensuite on va créer la methode qui lance le jeu, les differentes fonctionnalités et charge les évenements

In [ ]:
def update(self):
        self.draw_arena()
        self.ball.draw()
        self.paddles['user'].draw()
        self.paddles['computer'].draw()
        self.ball.move()
        self.paddles['computer'].move()
        if self.ball.hit_paddle_user(self.paddles['user']):
            self.ball.bounce('x')
            self.score.score += 1
        self.score.display(self.score.score)
        if self.ball.hit_paddle_computer(self.paddles['computer']):
            self.ball.bounce('x')


### Tests fonctionnels

On a essayer de definir la méthode de rebond de la balle
Dans un premier temps on a supposé que la balle rebondissait selon l'axe x mais suite à l'execution on a constaté que la balle traversait la fenetre de jeu
C'est pour cette raison qu'on a décidé de faire le rebond en fonction de y
D'ou le code;

In [ ]:
if self.ball.hit_paddle_user(self.paddles['user']):
            self.ball.bounce('y')
            self.score.score += 1
            self.score.display(self.score.score)
if self.ball.hit_paddle_computer(self.paddles['computer']):
            self.ball.bounce('y')

####Fonction Pause

Nous avons testé différents bouts de code pour faire la fonction pause, mais à chaque fois ils divisaient par 2 la frame rate
donc nous ne l'avons pas laissé:

In [ ]:
        if cv.waitKey(1) & 0xFF == ord('q'):    #waitKey(1) & 0xFF is a bitwise operation to only keep the last 8 bits and compare it to ord('q')
            break

        '''
        k = cv.waitKey(40)

        if k == ord(' '):
            paused = not paused
        '''

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
              
            #if event.type == pygame.KEYUP:
                if event.key == pygame.K_SPACE:
                    paused = not paused
        

        '''
        if cv.waitKey(1) & 0x70 == ord('p'):    #waitKey(1) & 0xFF is a bitwise operation to only keep the last 8 bits and compare it to ord('q')
            paused = not paused
        '''

### Bilan collectif et individuel





### Bibliographie (nos sources)
- https://code.visualstudio.com/docs/sourcecontrol/overview
- https://training.github.com/downloads/github-git-cheat-sheet.pdf
- https://stackoverflow.com/questions/7974849/how-can-i-make-one-python-file-run-another
- https://docplayer.fr/174641349-Introduction-a-pygame.html
- https://www.youtube.com/watch?v=lA5MHT1PbIA 
- https://stackoverflow.com/questions/35372700/whats-0xff-for-in-cv2-waitkey1 

